In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, DataType
import pyspark.sql.functions as fn
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
from operator import itemgetter
import json, datetime

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import datetime

env = "staging"
date_day= datetime.datetime.today().strftime("%Y%m%d")
hr = '1'
print(date_day)
s3_bucket = "s3://mist-{env}-kafka-reseed/ap-stats-full-{env}/".format(env=env)
s3_bucket += "{date}/{hr}/*/*/*.seqfile".format(date=date_day, hr=hr)

s3_path = s3_bucket 
print(s3_path)

20200404
s3://mist-staging-kafka-reseed/ap-stats-full-staging/20200404/1/*/*/*.seqfile


In [2]:
rdd = spark.sparkContext.sequenceFile(s3_path)

In [3]:
test= rdd.first()
test

(bytearray(b'\x08\x03\x10\x9d\x99\xcc\x05\x1a\x115c-5b-35-0e-ba-c9 \xb7\xd8\xcf\x95\x94.'),
 bytearray(b'\n\x06\\[5\x0e\xba\xc9\x12\x10\x18"o+\x02\x90L\xe6\xa3>\x052~}\rC"\x10\xbb\x7f\x98s\xc2\xf2E\xc8\x9dd\xce\x99j\xe5IM*\x0c\x08\xa5\xb8\x9f\xf4\x05\x10\xa7\xae\xe4\x99\x02\x8a\x01\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x90\x01\xdc\xdcH\x9a\x01\x07AP41-WW\xa2\x01\x1bapfw-0.5.17028-ellaria-ef58\xa8\x01\x01\xb9\x01_p\xec*{\x00N@\xc2\x01\x9b\x01\x12\x08\x08\xe4\xb6\x1f\x10\xd0\x8e\x1c\x1a\x1a\n\x18\n\x0bkmalloc-256\x10\x89\x8b\x01\x18\x90\x8c\x01 \x80\x02"\x10\n\x0c\x00\x00\xec<\x00\x00\xd1=\x00\x006>\x18U*\x002\x08\n\x06\x08\x80<\x10\xc8::U\n\x1b\n\x03cpu\x10\xdb\x84\xc0\x01 \xb2\xff\xed\x02(\xad\x91\xc0j0\x0b@\xaa\xbe\xdf\x02\x12\x19\n\x04cpu0\x10\xfe\xb8^ \xb0\xaa\xb6\x01(\xc3\xd7\x8f5@\xa6\xb1\xc2\x01\x12\x1b\n\x04cpu1\x10\xdd\xcba \x82\xd5\xb7\x01(\xea\xb9\xb050\n@\x83\x8d\x9d\x01\xca\x01*\n\x04eth0\x10\x01\x18\x01(\xe8\x07

In [4]:
# str(test[1], encoding='utf-8')
# bytearray(test[0], encoding='utf-8')

In [5]:
import apstats_pb2 as apstats

ap_stats = apstats.APStats()

# ap_stats.ParseFromString(bytes())

In [6]:
bytes(test[1])

b'\n\x06\\[5\x0e\xba\xc9\x12\x10\x18"o+\x02\x90L\xe6\xa3>\x052~}\rC"\x10\xbb\x7f\x98s\xc2\xf2E\xc8\x9dd\xce\x99j\xe5IM*\x0c\x08\xa5\xb8\x9f\xf4\x05\x10\xa7\xae\xe4\x99\x02\x8a\x01\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x90\x01\xdc\xdcH\x9a\x01\x07AP41-WW\xa2\x01\x1bapfw-0.5.17028-ellaria-ef58\xa8\x01\x01\xb9\x01_p\xec*{\x00N@\xc2\x01\x9b\x01\x12\x08\x08\xe4\xb6\x1f\x10\xd0\x8e\x1c\x1a\x1a\n\x18\n\x0bkmalloc-256\x10\x89\x8b\x01\x18\x90\x8c\x01 \x80\x02"\x10\n\x0c\x00\x00\xec<\x00\x00\xd1=\x00\x006>\x18U*\x002\x08\n\x06\x08\x80<\x10\xc8::U\n\x1b\n\x03cpu\x10\xdb\x84\xc0\x01 \xb2\xff\xed\x02(\xad\x91\xc0j0\x0b@\xaa\xbe\xdf\x02\x12\x19\n\x04cpu0\x10\xfe\xb8^ \xb0\xaa\xb6\x01(\xc3\xd7\x8f5@\xa6\xb1\xc2\x01\x12\x1b\n\x04cpu1\x10\xdd\xcba \x82\xd5\xb7\x01(\xea\xb9\xb050\n@\x83\x8d\x9d\x01\xca\x01*\n\x04eth0\x10\x01\x18\x01(\xe8\x070\x83\xec\xc3\x0b8\xf1\xe2\x11@\xb3\xf4\xb4[H\x86\xff6\xd0\x01 \xd8\x01\xb0\x06\xe0\x01\xb4j\xca\x01\x0

In [7]:
ap_stats.ParseFromString(bytes(test[0]))

33

In [8]:
! pip install protobuf
! pip install google-cloud-translate
# import json
# df = rdd.map(lambda x: json.loads(x[1])).toDF() #.map(lambda x: json.loads(x[1])).



In [9]:
# df.first()